# 1. 기초세팅

## 1-1) 필요 라이브러리 import

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 120)
pd.set_option('display.max_columns', 40)
pd.set_option('display.max_colwidth', 100)

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm # color map

plt.style.use('ggplot')
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

import datetime
from datetime import datetime
import calendar

from scipy import stats
from scipy.stats import skew, norm, probplot, boxcox, kurtosis

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, silhouette_score

import math


C:\Users\Kyeul\AppData\Local\Temp\ipykernel_27988\1565726540.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## 1-2) 데이터 로드

+ Onlinesales_info : 2019년도 한해 동안 발생한 온라인 거래와 관련된 정보
+ Customer_info : 고객 관련 정보
+ Discount_info : 할인 관련 정보
+ Marketing_info : 마케팅 비용 관련 정보
+ Tax_info : 세금 관련 정보

In [2]:
online_sales = pd.read_csv('data/Onlinesales_info.csv')
discount_coupon = pd.read_csv('data/Discount_info.csv')
customer_data = pd.read_csv('data/customer_info.csv')
marketing_spend = pd.read_csv('data/marketing_info.csv')
tax_amount = pd.read_csv('data/Tax_info.csv')

## 2. EDA
### <p style="color: rgb(153, 204, 255);"> 2-1. 데이터 기본정보 확인</p>
+ 타입 확인
+ 갯수 확인
+ 중복값 확인
+ 결측치 확인
+ 결측치 비율 확인
+ 고유값 확인

In [3]:
online_sales.apply(lambda x: [x.unique()])

,고객ID,거래ID,거래날짜,제품ID,제품카테고리,수량,평균금액,배송료,쿠폰상태
0,"[USER_1358, USER_0190, USER_0066, USER_0345, USER_0683, USER_0730, USER_0585, USER_1347, USER_07...","[Transaction_0000, Transaction_0001, Transaction_0002, Transaction_0003, Transaction_0004, Trans...","[2019-01-01, 2019-01-02, 2019-01-03, 2019-01-04, 2019-01-05, 2019-01-06, 2019-01-07, 2019-01-08,...","[Product_0981, Product_0904, Product_0203, Product_0848, Product_0854, Product_0880, Product_088...","[Nest-USA, Office, Apparel, Bags, Drinkware, Lifestyle, Notebooks & Journals, Headgear, Waze, Fu...","[1, 5, 15, 52, 31, 2, 3, 26, 10, 4, 6, 57, 103, 62, 12, 41, 30, 21, 206, 516, 14, 155, 34, 258, ...","[153.71, 2.05, 17.53, 16.5, 5.15, 3.08, 10.31, 9.27, 0.98, 1.99, 122.77, 81.5, 14.02, 10.72, 1.0...","[6.5, 102.79, 28.78, 8.7, 20.0, 17.96, 24.47, 35.96, 18.47, 74.74, 35.3, 20.78, 13.78, 122.74, 1...","[Used, Not Used, Clicked]"


In [4]:
online_sales.apply(lambda x: [x.unique()]).T

,0
고객ID,"[USER_1358, USER_0190, USER_0066, USER_0345, USER_0683, USER_0730, USER_0585, USER_1347, USER_07..."
거래ID,"[Transaction_0000, Transaction_0001, Transaction_0002, Transaction_0003, Transaction_0004, Trans..."
거래날짜,"[2019-01-01, 2019-01-02, 2019-01-03, 2019-01-04, 2019-01-05, 2019-01-06, 2019-01-07, 2019-01-08,..."
제품ID,"[Product_0981, Product_0904, Product_0203, Product_0848, Product_0854, Product_0880, Product_088..."
제품카테고리,"[Nest-USA, Office, Apparel, Bags, Drinkware, Lifestyle, Notebooks & Journals, Headgear, Waze, Fu..."
수량,"[1, 5, 15, 52, 31, 2, 3, 26, 10, 4, 6, 57, 103, 62, 12, 41, 30, 21, 206, 516, 14, 155, 34, 258, ..."
평균금액,"[153.71, 2.05, 17.53, 16.5, 5.15, 3.08, 10.31, 9.27, 0.98, 1.99, 122.77, 81.5, 14.02, 10.72, 1.0..."
배송료,"[6.5, 102.79, 28.78, 8.7, 20.0, 17.96, 24.47, 35.96, 18.47, 74.74, 35.3, 20.78, 13.78, 122.74, 1..."
쿠폰상태,"[Used, Not Used, Clicked]"


In [5]:
online_sales.apply(lambda x: [x.unique()]).T.squeeze()

고객ID      [USER_1358, USER_0190, USER_0066, USER_0345, USER_0683, USER_0730, USER_0585, USER_1347, USER_07...
거래ID      [Transaction_0000, Transaction_0001, Transaction_0002, Transaction_0003, Transaction_0004, Trans...
거래날짜      [2019-01-01, 2019-01-02, 2019-01-03, 2019-01-04, 2019-01-05, 2019-01-06, 2019-01-07, 2019-01-08,...
제품ID      [Product_0981, Product_0904, Product_0203, Product_0848, Product_0854, Product_0880, Product_088...
제품카테고리    [Nest-USA, Office, Apparel, Bags, Drinkware, Lifestyle, Notebooks & Journals, Headgear, Waze, Fu...
수량        [1, 5, 15, 52, 31, 2, 3, 26, 10, 4, 6, 57, 103, 62, 12, 41, 30, 21, 206, 516, 14, 155, 34, 258, ...
평균금액      [153.71, 2.05, 17.53, 16.5, 5.15, 3.08, 10.31, 9.27, 0.98, 1.99, 122.77, 81.5, 14.02, 10.72, 1.0...
배송료       [6.5, 102.79, 28.78, 8.7, 20.0, 17.96, 24.47, 35.96, 18.47, 74.74, 35.3, 20.78, 13.78, 122.74, 1...
쿠폰상태                                                                                [Used, Not Used, Clicked]
Name: 0, d

In [6]:
display(online_sales.apply(lambda x: x.unique()))
display(online_sales.apply(lambda x: x.unique().shape))
display(display(online_sales.apply(lambda x: x.unique().shape[0])))

고객ID      [USER_1358, USER_0190, USER_0066, USER_0345, USER_0683, USER_0730, USER_0585, USER_1347, USER_07...
거래ID      [Transaction_0000, Transaction_0001, Transaction_0002, Transaction_0003, Transaction_0004, Trans...
거래날짜      [2019-01-01, 2019-01-02, 2019-01-03, 2019-01-04, 2019-01-05, 2019-01-06, 2019-01-07, 2019-01-08,...
제품ID      [Product_0981, Product_0904, Product_0203, Product_0848, Product_0854, Product_0880, Product_088...
제품카테고리    [Nest-USA, Office, Apparel, Bags, Drinkware, Lifestyle, Notebooks & Journals, Headgear, Waze, Fu...
수량        [1, 5, 15, 52, 31, 2, 3, 26, 10, 4, 6, 57, 103, 62, 12, 41, 30, 21, 206, 516, 14, 155, 34, 258, ...
평균금액      [153.71, 2.05, 17.53, 16.5, 5.15, 3.08, 10.31, 9.27, 0.98, 1.99, 122.77, 81.5, 14.02, 10.72, 1.0...
배송료       [6.5, 102.79, 28.78, 8.7, 20.0, 17.96, 24.47, 35.96, 18.47, 74.74, 35.3, 20.78, 13.78, 122.74, 1...
쿠폰상태                                                                                [Used, Not Used, Clicked]
dtype: obj

,고객ID,거래ID,거래날짜,제품ID,제품카테고리,수량,평균금액,배송료,쿠폰상태
0,1468,25061,365,1145,20,151,546,267,3


고객ID       1468
거래ID      25061
거래날짜        365
제품ID       1145
제품카테고리       20
수량          151
평균금액        546
배송료         267
쿠폰상태          3
dtype: int64

None

In [7]:
display(online_sales.shape)
display(online_sales.shape[0])

(52924, 9)

52924

In [8]:
def str_summary(df, pred=None):
    obs = df.shape[0]
    types = df.dtypes
    counts = df.apply(lambda x: x.count())
    uniques = df.apply(lambda x: [x.unique()]).T.squeeze() # 각 feature의 고유값을 시리즈 타입으로 반환
    nulls = df.apply(lambda x: x.isnull().sum())
    distincts = df.apply(lambda x: x.unique().shape[0]) # 각 feature의 고유값 개수
    missing_ratio = (df.isnull().sum() / obs) * 100
    
    print("Data shape: ", df.shape)

    cols = ['Types', 'Counts', 'Distinctse', 'Nulls', 'Missing_ratio', 'Uniques']
    structure = pd.concat([types, counts, distincts, nulls, missing_ratio, uniques], axis=1, sort=True)

    structure.columns = cols

    print("================================================")
    print("Data types: ")
    print(structure['Types'].value_counts())
    print("================================================")
    print("\n\n")

    return structure

In [9]:
data_summary_1 = str_summary(online_sales)
data_summary_2 = str_summary(discount_coupon)
data_summary_3 = str_summary(customer_data)
data_summary_4 = str_summary(marketing_spend)
data_summary_5 = str_summary(tax_amount)

Data shape:  (52924, 9)
Data types: 
Types
object     6
float64    2
int64      1
Name: count, dtype: int64



Data shape:  (204, 4)
Data types: 
Types
object    3
int64     1
Name: count, dtype: int64



Data shape:  (1468, 4)
Data types: 
Types
object    3
int64     1
Name: count, dtype: int64



Data shape:  (365, 3)
Data types: 
Types
object     1
int64      1
float64    1
Name: count, dtype: int64



Data shape:  (20, 2)
Data types: 
Types
float64    1
object     1
Name: count, dtype: int64





In [10]:
display(data_summary_1.sort_values(by='Missing_ratio', ascending=False))
display(data_summary_2.sort_values(by='Missing_ratio', ascending=False))
display(data_summary_3.sort_values(by='Missing_ratio', ascending=False))
display(data_summary_4.sort_values(by='Missing_ratio', ascending=False))
display(data_summary_5.sort_values(by='Missing_ratio', ascending=False))

,Types,Counts,Distinctse,Nulls,Missing_ratio,Uniques
거래ID,object,52924,25061,0,0.0,"[Transaction_0000, Transaction_0001, Transaction_0002, Transaction_0003, Transaction_0004, Trans..."
거래날짜,object,52924,365,0,0.0,"[2019-01-01, 2019-01-02, 2019-01-03, 2019-01-04, 2019-01-05, 2019-01-06, 2019-01-07, 2019-01-08,..."
고객ID,object,52924,1468,0,0.0,"[USER_1358, USER_0190, USER_0066, USER_0345, USER_0683, USER_0730, USER_0585, USER_1347, USER_07..."
배송료,float64,52924,267,0,0.0,"[6.5, 102.79, 28.78, 8.7, 20.0, 17.96, 24.47, 35.96, 18.47, 74.74, 35.3, 20.78, 13.78, 122.74, 1..."
수량,int64,52924,151,0,0.0,"[1, 5, 15, 52, 31, 2, 3, 26, 10, 4, 6, 57, 103, 62, 12, 41, 30, 21, 206, 516, 14, 155, 34, 258, ..."
제품ID,object,52924,1145,0,0.0,"[Product_0981, Product_0904, Product_0203, Product_0848, Product_0854, Product_0880, Product_088..."
제품카테고리,object,52924,20,0,0.0,"[Nest-USA, Office, Apparel, Bags, Drinkware, Lifestyle, Notebooks & Journals, Headgear, Waze, Fu..."
쿠폰상태,object,52924,3,0,0.0,"[Used, Not Used, Clicked]"
평균금액,float64,52924,546,0,0.0,"[153.71, 2.05, 17.53, 16.5, 5.15, 3.08, 10.31, 9.27, 0.98, 1.99, 122.77, 81.5, 14.02, 10.72, 1.0..."


,Types,Counts,Distinctse,Nulls,Missing_ratio,Uniques
월,object,204,12,0,0.0,"[Jan, Feb, Mar, Apr, May, Jun, Jul, Aug, Sep, Oct, Nov, Dec]"
제품카테고리,object,204,17,0,0.0,"[Apparel, Nest-USA, Office, Drinkware, Lifestyle, Bags, Notebooks, Headgear, Nest, Waze, Bottles..."
쿠폰코드,object,204,48,0,0.0,"[SALE10, SALE20, SALE30, ELEC10, ELEC20, ELEC30, OFF10, OFF20, OFF30, EXTRA10, EXTRA20, EXTRA30,..."
할인율,int64,204,3,0,0.0,"[10, 20, 30]"


,Types,Counts,Distinctse,Nulls,Missing_ratio,Uniques
가입기간,int64,1468,49,0,0.0,"[12, 43, 33, 30, 49, 32, 46, 24, 40, 10, 19, 14, 25, 50, 39, 21, 29, 26, 41, 28, 15, 18, 31, 27,..."
고객ID,object,1468,1468,0,0.0,"[USER_1358, USER_0190, USER_0066, USER_0345, USER_0683, USER_0730, USER_0585, USER_1347, USER_07..."
고객지역,object,1468,5,0,0.0,"[Chicago, California, New York, New Jersey, Washington DC]"
성별,object,1468,2,0,0.0,"[남, 여]"


,Types,Counts,Distinctse,Nulls,Missing_ratio,Uniques
날짜,object,365,365,0,0.0,"[2019-01-01, 2019-01-02, 2019-01-03, 2019-01-04, 2019-01-05, 2019-01-06, 2019-01-07, 2019-01-08,..."
오프라인비용,int64,365,11,0,0.0,"[4500, 5000, 3000, 1000, 700, 3500, 4000, 2500, 2000, 500, 1500]"
온라인비용,float64,365,365,0,0.0,"[2424.5, 3480.36, 1576.38, 2928.55, 4055.3, 3796.85, 2579.52, 2551.38, 3287.83, 515.44, 1757.58,..."


,Types,Counts,Distinctse,Nulls,Missing_ratio,Uniques
GST,float64,20,4,0,0.0,"[0.1, 0.18, 0.05, 0.12]"
제품카테고리,object,20,20,0,0.0,"[Nest-USA, Office, Apparel, Bags, Drinkware, Lifestyle, Notebooks & Journals, Headgear, Waze, Fu..."


In [11]:
display(online_sales.sample(15))
display(discount_coupon.sample(15))
display(customer_data.sample(15))
display(marketing_spend.sample(15))
display(tax_amount.sample(15))


,고객ID,거래ID,거래날짜,제품ID,제품카테고리,수량,평균금액,배송료,쿠폰상태
23640,USER_0594,Transaction_11066,2019-06-23,Product_0976,Nest-USA,2,119.00,6.00,Used
26374,USER_0112,Transaction_12311,2019-07-13,Product_0970,Office,1,1.99,6.00,Used
52339,USER_0923,Transaction_24659,2019-12-24,Product_0990,Nest,2,202.84,12.99,Clicked
38700,USER_0021,Transaction_17164,2019-09-20,Product_0976,Nest-USA,1,119.00,6.00,Used
5346,USER_0952,Transaction_2781,2019-02-12,Product_0293,Apparel,1,16.99,6.50,Used
3414,USER_0800,Transaction_1782,2019-01-27,Product_0294,Apparel,5,14.02,35.96,Clicked
11302,USER_1323,Transaction_5587,2019-03-29,Product_0658,Apparel,1,44.79,6.50,Clicked
19174,USER_1028,Transaction_9077,2019-05-23,Product_0222,Apparel,1,15.99,6.00,Used
52821,USER_1164,Transaction_24994,2019-12-30,Product_0989,Nest-USA,1,151.88,6.50,Clicked
16580,USER_0715,Transaction_7927,2019-05-04,Product_1137,Apparel,1,10.99,6.00,Used


,월,제품카테고리,쿠폰코드,할인율
132,Jul,Bottles,BT10,10
9,Jan,Drinkware,EXTRA10,10
116,Sep,Lifestyle,EXTRA30,30
70,May,Notebooks,NOTES20,20
17,Mar,Bags,AIO30,30
35,Mar,Nest-Canada,NCA30,30
12,Jan,Lifestyle,EXTRA10,10
110,Sep,Office,OFF30,30
200,Dec,Notebooks & Journals,NJ30,30
66,Apr,Bags,AIO10,10


,고객ID,성별,고객지역,가입기간
285,USER_0713,여,California,7
494,USER_0980,남,California,19
786,USER_0283,남,Chicago,38
1243,USER_1043,여,New Jersey,3
1001,USER_1019,여,California,10
1305,USER_1115,남,Washington DC,47
331,USER_0480,여,California,18
759,USER_1084,여,Chicago,15
910,USER_0737,여,Chicago,47
371,USER_1053,여,New York,30


,날짜,오프라인비용,온라인비용
141,2019-05-22,1000,979.08
214,2019-08-03,1500,2493.64
131,2019-05-12,3000,2113.63
99,2019-04-10,3000,2143.39
104,2019-04-15,3000,1593.58
68,2019-03-10,2500,1294.22
232,2019-08-21,2500,1528.10
313,2019-11-10,3500,2431.73
171,2019-06-21,3500,1473.98
301,2019-10-29,2000,1564.09


,제품카테고리,GST
1,Office,0.10
10,Nest-Canada,0.10
11,Backpacks,0.10
3,Bags,0.18
15,More Bags,0.18
13,Bottles,0.05
18,Accessories,0.10
5,Lifestyle,0.18
0,Nest-USA,0.10
19,Nest,0.05


5개 데이터셋 모두 결측치가 포함되어 있지 않다. 

online_sales 특이사항
+ [고객ID]는 중복값이 포함되어 있다. 즉, 한명의 고객이 여러번 구매했다.
+ [거래ID]는 중복값이 포함되어 있고, 하나의 거래 ID에 서로 다른 고객 ID가 포함되어 있다. 그리고 중복 거래ID는 모두 같은 날자에 발생하였다. 하지만 같은 [거래날짜]라고 [거래ID] 같은 것은 아니다. 하루 안에서 비슷한 시간대?에 구매가 이뤄졌거나 다른 이유가 있을 것 같다. (공구 같은 서비스가 있었나..?)
+ [쿠폰상태]의 Clicked는 쿠폰에 관심이 있어 클릭했지만 결제까지 연결되지 못한 데이터라고 판단됨.

In [12]:
online_sales['거래ID'].value_counts()

거래ID
Transaction_12261    35
Transaction_4716     30
Transaction_19047    29
Transaction_13487    28
Transaction_16759    27
                     ..
Transaction_10305     1
Transaction_10306     1
Transaction_10309     1
Transaction_10312     1
Transaction_25060     1
Name: count, Length: 25061, dtype: int64

In [16]:
# 같은 거래ID인 데이터 살펴보기
display(online_sales[online_sales['거래ID']=='Transaction_12261'].sample(10))
display(online_sales[online_sales['거래ID']=='Transaction_4716'].sample(10))
display(online_sales[online_sales['거래ID']=='Transaction_13487'].sample(10))

,고객ID,거래ID,거래날짜,제품ID,제품카테고리,수량,평균금액,배송료,쿠폰상태
26232,USER_0118,Transaction_12261,2019-07-13,Product_0728,Apparel,7,16.79,6.0,Clicked
26237,USER_1391,Transaction_12261,2019-07-13,Product_0739,Apparel,76,23.79,6.0,Used
26210,USER_0050,Transaction_12261,2019-07-13,Product_0155,Apparel,26,11.89,6.0,Clicked
26233,USER_1391,Transaction_12261,2019-07-13,Product_0729,Apparel,11,16.79,6.0,Used
26227,USER_0118,Transaction_12261,2019-07-13,Product_0723,Apparel,22,26.59,6.0,Used
26229,USER_0118,Transaction_12261,2019-07-13,Product_0725,Apparel,20,16.79,6.0,Not Used
26220,USER_0118,Transaction_12261,2019-07-13,Product_0167,Apparel,16,13.29,6.0,Clicked
26212,USER_0050,Transaction_12261,2019-07-13,Product_0157,Apparel,5,11.89,6.0,Clicked
26231,USER_0118,Transaction_12261,2019-07-13,Product_0727,Apparel,1,16.79,6.0,Clicked
26209,USER_0050,Transaction_12261,2019-07-13,Product_0154,Apparel,5,11.89,6.0,Not Used


,고객ID,거래ID,거래날짜,제품ID,제품카테고리,수량,평균금액,배송료,쿠폰상태
9354,USER_0187,Transaction_4716,2019-03-15,Product_0203,Apparel,1,13.59,6.5,Clicked
9374,USER_0187,Transaction_4716,2019-03-15,Product_0920,Headgear,1,19.99,6.5,Clicked
9381,USER_1178,Transaction_4716,2019-03-15,Product_0971,Lifestyle,5,1.20,6.5,Clicked
9377,USER_0187,Transaction_4716,2019-03-15,Product_0940,Office,20,0.79,6.5,Clicked
9373,USER_0187,Transaction_4716,2019-03-15,Product_0918,Lifestyle,5,2.80,6.5,Not Used
9365,USER_0187,Transaction_4716,2019-03-15,Product_0889,Google,1,19.99,6.5,Not Used
9372,USER_0187,Transaction_4716,2019-03-15,Product_0917,Lifestyle,5,2.80,6.5,Not Used
9364,USER_0187,Transaction_4716,2019-03-15,Product_0885,Drinkware,5,7.99,6.5,Used
9380,USER_1178,Transaction_4716,2019-03-15,Product_0966,Office,15,1.59,6.5,Used
9360,USER_0187,Transaction_4716,2019-03-15,Product_0843,Bags,5,11.19,6.5,Clicked


,고객ID,거래ID,거래날짜,제품ID,제품카테고리,수량,평균금액,배송료,쿠폰상태
29494,USER_0312,Transaction_13487,2019-07-30,Product_0442,Apparel,2,5.10,6.0,Clicked
29501,USER_0312,Transaction_13487,2019-07-30,Product_0538,Apparel,2,5.70,6.0,Clicked
29488,USER_0350,Transaction_13487,2019-07-30,Product_0246,Apparel,3,6.00,6.0,Clicked
29497,USER_0312,Transaction_13487,2019-07-30,Product_0534,Apparel,2,5.70,6.0,Clicked
29500,USER_0312,Transaction_13487,2019-07-30,Product_0537,Apparel,2,5.70,6.0,Clicked
29493,USER_0312,Transaction_13487,2019-07-30,Product_0441,Apparel,2,5.10,6.0,Used
29484,USER_0350,Transaction_13487,2019-07-30,Product_0038,Apparel,2,6.00,6.0,Clicked
29506,USER_0205,Transaction_13487,2019-07-30,Product_0942,Office,20,2.50,6.0,Clicked
29503,USER_0312,Transaction_13487,2019-07-30,Product_0843,Bags,14,13.99,6.0,Clicked
29504,USER_0312,Transaction_13487,2019-07-30,Product_0854,Bags,20,4.99,6.0,Clicked


In [17]:
# 같은 날짜에 발생한 것들은 같은 거래ID를 가질까? 
online_sales[online_sales['거래날짜']=='2019-07-30']

,고객ID,거래ID,거래날짜,제품ID,제품카테고리,수량,평균금액,배송료,쿠폰상태
29437,USER_0350,Transaction_13466,2019-07-30,Product_0976,Nest-USA,3,119.00,6.00,Clicked
29438,USER_0350,Transaction_13466,2019-07-30,Product_0983,Nest-USA,2,119.00,6.00,Used
29439,USER_0350,Transaction_13467,2019-07-30,Product_0854,Bags,1,4.99,6.00,Not Used
29440,USER_0350,Transaction_13468,2019-07-30,Product_0983,Nest-USA,2,119.00,19.99,Clicked
29441,USER_0350,Transaction_13469,2019-07-30,Product_0984,Nest-USA,1,79.00,6.00,Not Used
...,...,...,...,...,...,...,...,...,...
29563,USER_1411,Transaction_13510,2019-07-30,Product_0448,Apparel,1,5.10,6.00,Clicked
29564,USER_1411,Transaction_13510,2019-07-30,Product_0536,Apparel,1,5.70,6.00,Used
29565,USER_1411,Transaction_13510,2019-07-30,Product_0537,Apparel,1,5.70,6.00,Clicked
29566,USER_1411,Transaction_13511,2019-07-30,Product_0981,Nest-USA,1,149.00,6.00,Used


## 2-2) 데이터 통합을 위한 전처리

In [ ]:
df = online_sales.copy()

# customers_data 통합
df['거래날짜'] = pd.to_datetime(df['거래날짜'])
df = df.merge(customer_data, on="고객ID")

# discount_coupon 통합
df['월'] = df['거래날짜'].dt.month
month_dict = {
    
}